In [40]:
import numpy as np

In [34]:
def durbin_levinson(autocovariance, nlags):
    """
    Estimates the coefficients of an time series model using the Durbin-Levinson algorithm.

    Args:
        - autocovariance: array-like
        1D array containing the autocovariance values at lags 0 to `nlags`.

        - nlags: int
        Number of lags to consider for the AR model.

    Returns:
        - coefficients: array-like
        - 1D array of estimated AR coefficients of order `nlags`.

    """
    # Extract array lengths
    n_terms = len(autocovariance)

    # Initialize memory variables
    coefficients = np.zeros(n_terms)  # Stores final coefficients (phi_nn)
    forward_errors = np.zeros(n_terms)  # Stores forward prediction errors (v_n)
    reflection_coefficients = np.zeros(n_terms)  # Stores reflection coefficients

    # Initial values
    coefficients[0] = 1.0
    forward_errors[0] = autocovariance[0]

    # Iterate through lags
    for i in range(1, nlags + 1):
        # Calculate reflection coefficient
        reflection_coefficients[i] = (autocovariance[i] - np.dot(coefficients[1:i], autocovariance[i-1:0:-1])) / forward_errors[i-1]

        # Update model coefficients
        coefficients[i] = reflection_coefficients[i]
        coefficients[1:i] -= reflection_coefficients[i] * coefficients[i-1:0:-1]

        # Update forward prediction error
        forward_errors[i] = forward_errors[i-1] * (1.0 - reflection_coefficients[i]**2)

    return coefficients[:nlags + 1]  # Return estimated AR coefficients of order `nlags`


In [39]:
print(f"Model parameter for prediction: {durbin_levinson([1.81, -0.9, 0, 0, 0], 4)[1:]}")

Model parameter for prediction: [-0.78698379 -0.58271185 -0.38491448 -0.19139394]


# Test the result

In [22]:
import statsmodels
from statsmodels.tsa.stattools import levinson_durbin

sigma_v, arcoefs, pacf, sigma, phi = statsmodels.tsa.stattools.levinson_durbin(np.array([1.81, -0.9, 0, 0, 0]), nlags=4, isacov=True)

## The partial autocorrelation function.

In [15]:
pacf

array([ 1.        , -0.49723757, -0.32845383, -0.24319934, -0.19139394])

## The estimate of the error variance.

In [16]:
sigma_v

1.1017145872296494

## The estimate of the autoregressive coefficients for a model including nlags.

In [23]:
arcoefs

array([-0.74043691, -0.4891009 , -0.24319934])

## The entire sigma array from intermediate result, last value is sigma_v.

In [18]:
sigma

array([0.        , 1.36248619, 1.21549856, 1.14360678, 1.10171459])

## The entire phi array from intermediate result, last column contains autoregressive coefficients for AR(nlags).

In [21]:
phi.T

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.49723757,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.66055716, -0.32845383,  0.        ,  0.        ],
       [ 0.        , -0.74043691, -0.4891009 , -0.24319934,  0.        ],
       [ 0.        , -0.78698379, -0.58271185, -0.38491448, -0.19139394]])